In [1]:
import pandas as pd

# pandas 缺失值简介

## 缺失值的表现形式

In [1]:
import numpy as np
print(np.NaN)
print(np.NAN)
print(np.nan)

nan
nan
nan


In [2]:
print(np.NaN==True)
print(np.NaN==False)
print(np.NaN==0)
print(np.NaN=='')
print(np.NaN==None)

False
False
False
False
False


In [3]:
print(np.NaN==np.NaN)
print(np.NaN==np.nan)
print(np.NaN==np.NAN)
print(np.NAN==np.nan)

False
False
False
False


## 缺失值判断

**pandas 判断是否为缺失值方法**：
pd.isnull和pd.notnull

**判断 Series 中的元素是否是缺失值**：series.isnull()

**判断 DataFrame 中的数据是否是缺失值**：df.isnull()

## 加载包含缺失值的数据

**加载包含缺失值的数据：survey_visited.csv**

**加载数据时，不显示默认的缺失值，默认缺失值填充为''**

**na_values=["DR-3"]：加载数据时，把 'DR-3' 当做缺失值**

# pandas 缺失值处理

##  加载数据并查看缺失情况

### 数据基本缺失情况查看

**加载包含缺失值的数据：titanic_train.csv**

### 构造缺失值统计的函数

### 使用Missingno库对缺失值的情况进行可视化探查